In [7]:
### PREREQUISTIES
### (many used only in one notebook...)

import os
import pandas as pd
import numpy as np
import json
import logging ### to monitor the code
from bs4 import BeautifulSoup
import pickle
import math
import random
import sys
import csv
import unicodedata
import requests
from urllib.request import urlopen 
import io
import getpass
import re
from collections import defaultdict

import nltk
from nltk.collocations import *

import networkx as nx

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

import xml.etree.cElementTree as ET
from urllib.request import urlopen

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe

!pip install sddk
import sddk

!pip install anda
from anda import gr

In [8]:
conf = sddk.configure("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ··········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [9]:
sddk.list_filenames("SDAM_data/AGT", conf=conf)

['..',
 'AGT.json',
 'GR1000_metadata.json',
 'LG_data_cleaned_20191020.json',
 'MorpheusUnicode.xml',
 'author_cent_prov.json',
 'data_merged_dated.json',
 'data_merged_lem.json',
 'data_merged_raw.json',
 'data_ogl.json',
 'data_perseus.json',
 'morpheus_by_lemma.json',
 'morpheus_dict.json',
 'ogl_paths.json',
 'perseus_paths.json',
 'perseus_paths2.json',
 'proxies.json',
 'theos_results.json',
 'theos_results_20200220.json',
 'tlg_authordate.json',
 'tlg_authordate_cleaned.json',
 'tlg_authors_with_dating_probs.json',
 'works_overview.json',
 'http://sabre.io/']

In [10]:
data_merged_raw = sddk.read_file("SDAM_data/AGT/data_merged_raw.json", "df", conf)
data_merged_raw.head(5)

,filename,author,title,string
0,ggm0001.ggm001.1st1K-grc1.xml,Anonymous,Anametresis Pontou; Geographi Graeci Minores,ΑΝΩΝΥΜΟΥ ΑΝΑΜΕΤΡΗΣΙΣ ΤΗΣ ΟΙΚΟΥΜΕΝΗΣ ΗΑΣ ΗΕ ΚΑΤ...
1,heb0001.heb010.1st1K-eng1.xml,None,Isaias; The Book of Isaiah,ISAIAH FROM HEBREW I. 1 The vision of Isaiah t...
2,ogl0001.ogl001.1st1K-grc1.xml,Pinytus,De Epistola Pinyti ad Dionysium; Reliquiae sacrae,"FRAGMENTUM BEATI PINYTI, CNOSSI IN CRETA EPISC..."
3,stoa0033a.tlg028.1st1K-grc1.xml,pseudo-Aristotle,De mundo; Aristotelis Opera,ΠΕΡΙ ΚΟΣΜΟΥ ΠΡΟΣ ΑΛΕΞΑΝΔΡΟΝ. ΠΟΛΛΑΚΙΣ μὲν ἔμοι...
4,stoa0033a.tlg043.1st1K-grc1.xml,pseudo-Aristotle,De spiritu; Aristotelis Opera,ΠΕΡΙ ΠΝΕΥΜΑΤΟΣ. ΤΙΣ ἡ τοῦ ἐμφύτου πνεύματος δι...


# Testing lemmatization and translation

In [11]:
test_string = data_merged_raw[data_merged_raw["filename"].str.startswith("tlg0086.")]["string"].tolist()[0]
print(test_string)

ΑΝΑΛΥΤΙΚΩΝ ΠΡΟΤΕΡΩΝ Α. ΠΡΩΤΟΝ εἰπεῖν περὶ τί καὶ τίνος ἐστὶν ἡ σκέψις, ὅτι περὶ ἀπόδειξιν καὶ ἐπιστήμης ἀποδεικτικῆς· εἶτα διορίσαι τί ἐστι πρότασις καὶ τί ὅρος καὶ τί συλλογισμός, καὶ ποῖος τέλειος καὶ ποῖος ἀτελής, μετὰ δὲ ταῦτα τί τὸ ἐν ὅλῳ εἶναι ἢ μὴ εἶναι τόδε τῷδε, καὶ τί λέγομεν τὸ κατὰ παντὸς ἢ μηδενὸς κατηγορεῖσθαι. Πρότασις μὲν οὖν ἐστὶ λόγος καταφατικὸς ἢ ἀποφατικὸς τινὸς κατά τινος. Οὗτος δὲ ἢ καθόλου ἢ ἐν μέρει ἢ ἀδιόριστος. Λέγω δὲ καθόλου μὲν τὸ παντὶ ἢ μηδενὶ ὑπάρχειν, ἐν μέρει δὲ τὸ τινὶ ἢ μὴ τινὶ ἢ μὴ παντὶ ὑπάρχειν, ἀδιόριστον δὲ τὸ ὑπάρχειν ἢ μὴ ὑπάρχειν ἄνευ τοῦ καθόλου, ἢ κατὰ μέρος, οἷον τὸ τῶν ἐναντίων εἶναι τὴν αὐτὴν ἐπιστήμην ἢ τὸ τὴν ἡδονὴν μὴ εἶναι ἀγαθόν. Διαφέρει δὲ ἡ ἀποδεικτικὴ πρότασις τῆς διαλεκτικῆς, ὅτι ἡ μὲν ἀποδεικτικὴ λῆψις θατέρου μορίου τῆς ἀντιφάσεώς ἐστιν (οὐ γὰρ ἐρωτᾷ ἀλλὰ λαμβάνει ὁ ἁποδεικνύων), ἡ δὲ διαλεκτικὴ ἐρώτησις ἀντιφάσεώς ἐστιν. Οὐδὲν δὲ διοίσει πρὸς τὸ γενέσθαι τὸν ἑκατέρου συλλογισμόν· καὶ γὰρ ὁ ἀποδεικνύων καὶ ὁ ἐρωτῶν συλλογίζε

In [14]:
gr.get_lemmatized_sentences(test_string, all_lemmata=False, involve_unknown=True, filter_by_postag=["n","a","v"])[:3]

[['αναλυτικων', 'προτερων'],
 ['πρωτον', 'εἶπον', 'σκέψις', 'ἀπόδειξις', 'ἐπιστήμη', 'ἀποδεικτικῆς'],
 ['διορίζω',
  'πρότασις',
  'ὅρος',
  'συλλογισμός',
  'ποιός',
  'τέλειος',
  'ποιός',
  'ἀτελής',
  'ὁλάω',
  'λέγω',
  'πᾶς',
  'μηδείς',
  'κατηγορέω']]

# Get Lemmatized Sentences for the Whole Corpus

In [15]:
%%time
data_merged_raw["lem_senteces"] = data_merged_raw.apply(lambda row: gr.get_lemmatized_sentences(row["string"], all_lemmata=False, involve_unknown=True, filter_by_postag=["n","a","v"]), axis=1)

CPU times: user 3min 26s, sys: 712 ms, total: 3min 27s
Wall time: 3min 27s


In [16]:
data_merged_raw.head(5)

,filename,author,title,string,lem_senteces
0,ggm0001.ggm001.1st1K-grc1.xml,Anonymous,Anametresis Pontou; Geographi Graeci Minores,ΑΝΩΝΥΜΟΥ ΑΝΑΜΕΤΡΗΣΙΣ ΤΗΣ ΟΙΚΟΥΜΕΝΗΣ ΗΑΣ ΗΕ ΚΑΤ...,"[[ανωνυμου, αναμετρησις, της, οικουμενης, ηας,..."
1,heb0001.heb010.1st1K-eng1.xml,None,Isaias; The Book of Isaiah,ISAIAH FROM HEBREW I. 1 The vision of Isaiah t...,"[[μάταιος], [ιο], [Πάρος, ἐμέω], [ἃς], [ěā], [..."
2,ogl0001.ogl001.1st1K-grc1.xml,Pinytus,De Epistola Pinyti ad Dionysium; Reliquiae sacrae,"FRAGMENTUM BEATI PINYTI, CNOSSI IN CRETA EPISC...","[[ἣν, πινυτός, ἀντιγραφή, θαυμάζω, ἀποδέχομαι,..."
3,stoa0033a.tlg028.1st1K-grc1.xml,pseudo-Aristotle,De mundo; Aristotelis Opera,ΠΕΡΙ ΚΟΣΜΟΥ ΠΡΟΣ ΑΛΕΞΑΝΔΡΟΝ. ΠΟΛΛΑΚΙΣ μὲν ἔμοι...,"[[περι, κοσμου, προς, αλεξανδρον], [πολλακις, ..."
4,stoa0033a.tlg043.1st1K-grc1.xml,pseudo-Aristotle,De spiritu; Aristotelis Opera,ΠΕΡΙ ΠΝΕΥΜΑΤΟΣ. ΤΙΣ ἡ τοῦ ἐμφύτου πνεύματος δι...,"[[περι, πνευματος], [ἔμφυτος, πνεῦμα, διαμονή,..."


In [18]:
sddk.write_file("SDAM_data/AGT/data_merged_lem.json", data_merged_raw, conf)

A file with the same name ("data_merged_lem.json") already exists in this location.
Press Enter to overwrite it or choose different path and filename: 
Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/AGT/data_merged_lem.json"


In [0]:
[(word, return_all_unique_translations(word)) for word in tokenize_string(test_string)]

[('ΑΝΑΛΥΤΙΚΩΝ', ''),
 ('ΠΡΟΤΕΡΩΝ',
  'prae. / to be before, be in advance / to speak to, address, accost'),
 ('ΠΡΩΤΟΝ', 'prae.'),
 ('εἰπεῖν', 'to speak, say'),
 ('τίνος', 'any one, any thing, some one, some thing; / who? which?'),
 ('ἐστίν', 'I have'),
 ('σκέψις', 'a viewing, perception by the senses'),
 ('ἀπόδειξιν', 'a shewing forth, exhibiting'),
 ('ἐπιστήμης', 'acquaintance with'),
 ('ἀποδεικτικῆς', ''),
 ('εἶτα', 'then, next'),
 ('διορίσαι', 'to draw a boundary through, divide by limits, separate'),
 ('ἐστι', 'I have'),
 ('πρότασις', 'a proposition, the premiss'),
 ('ὅρος', 'a boundary, landmark'),
 ('συλλογισμός', 'computation'),
 ('ποῖος', 'of a certain nature, kind / of what nature? of what sort?'),
 ('τέλειος', 'having reached its end, finished, complete'),
 ('ποῖος', 'of a certain nature, kind / of what nature? of what sort?'),
 ('ἀτελής', 'without end'),
 ('ταῦτα', 'this'),
 ('ὅλῳ', ''),
 ('εἶναι', 'I have'),
 ('εἶναι', 'I have'),
 ('τόδε', 'this'),
 ('τῷδε', 'this'),
 ('λέγ

In [0]:
print(lemmatize_string(test_string, all_lemmata=True, filter_by_postag=["n","a","v"], involve_unknown=True)[:100])

['αναλυτικων', 'πρότερος', 'πρότερος', 'εἶπον', 'σκέψις', 'ἀπόδειξις', 'ἐπιστήμη', 'ἀποδεικτικῆς', 'διορίζω', 'πρότασις', 'ὅρος', 'συλλογισμός', 'ποῖος/ποιός', 'τέλειος', 'ποῖος/ποιός', 'ἀτελής', 'ὅλοξ/ὁλάω', 'λέγω', 'πᾶς', 'μηδείς', 'κατηγορέω', 'πρότασις', 'λόγος', 'καταφατικός', 'ἀποφατικός', 'μέρος', 'ἀδιόριστος', 'λέγω', 'πᾶς', 'μηδείς', 'ὑπάρχω', 'μέρος', 'πᾶς', 'ὑπάρχω', 'ἀδιόριστον', 'ὑπάρχω', 'ὑπάρχω', 'ἀνίημι/ἄνω', 'μέρος', 'οἷος', 'ἐναντίος', 'αὐτην', 'ἐπιστήμη', 'ἡδονην', 'ἀγαθός', 'διαφέρω', 'ἀποδεικτικη', 'πρότασις', 'διαλεκτικός', 'ἀποδεικτικη', 'λῆψις', 'ἕτερος', 'μόριος/μόριον', 'ἀντιφάσεώς', 'ἐρωτάω', 'λαμβάνω', 'ἁποδεικνύων', 'διαλεκτικός', 'ἐρώτησις', 'ἀντιφάσεώς', 'διαφέρω', 'γίγνομαι', 'ἑκάτερος', 'συλλογισμός', 'ἔρως/ἐρωτάω', 'συλλογίζομαι', 'λαβή', 'ὑπάρχω', 'ὑπάρχω', 'συλλογιστικη', 'πρότασις', 'ἁπλόω', 'κατάφασις', 'ἀπόφασίς', 'τρόπος/τροπός', 'ἀποδεικτικη', 'ἀληθης', 'ἀρχή', 'ὑπόθεσις', 'εἰλημμένη', 'διαλεκτικός', 'ἐρώτησις', 'ἀντιφάσεως', 'συλλογιζομένῳ', 'λ

In [0]:
word = "ἀποδεικτικῆς"
return_list_of_tokens(word)

[{'a': '',
  'b': '',
  'd': '',
  'e': '',
  'f': 'ἀποδεικτικῆς',
  'i': '',
  'l': 'ἀποδεικτικῆς',
  'p': '',
  's': ''}]

In [0]:
sentences = []
for string in data_merged_raw[data_merged_raw["filename"].str.startswith("tlg0086.")]["string"].tolist()[:5]:
  sentences.extend(get_sentences(string))
print(sentences)

['ΑΝΑΛΥΤΙΚΩΝ ΠΡΟΤΕΡΩΝ Α', 'ΠΡΩΤΟΝ εἰπεῖν περὶ τί καὶ τίνος ἐστὶν ἡ σκέψις, ὅτι περὶ ἀπόδειξιν καὶ ἐπιστήμης ἀποδεικτικῆς', 'εἶτα διορίσαι τί ἐστι πρότασις καὶ τί ὅρος καὶ τί συλλογισμός, καὶ ποῖος τέλειος καὶ ποῖος ἀτελής, μετὰ δὲ ταῦτα τί τὸ ἐν ὅλῳ εἶναι ἢ μὴ εἶναι τόδε τῷδε, καὶ τί λέγομεν τὸ κατὰ παντὸς ἢ μηδενὸς κατηγορεῖσθαι', 'Πρότασις μὲν οὖν ἐστὶ λόγος καταφατικὸς ἢ ἀποφατικὸς τινὸς κατά τινος', 'Οὗτος δὲ ἢ καθόλου ἢ ἐν μέρει ἢ ἀδιόριστος', 'Λέγω δὲ καθόλου μὲν τὸ παντὶ ἢ μηδενὶ ὑπάρχειν, ἐν μέρει δὲ τὸ τινὶ ἢ μὴ τινὶ ἢ μὴ παντὶ ὑπάρχειν, ἀδιόριστον δὲ τὸ ὑπάρχειν ἢ μὴ ὑπάρχειν ἄνευ τοῦ καθόλου, ἢ κατὰ μέρος, οἷον τὸ τῶν ἐναντίων εἶναι τὴν αὐτὴν ἐπιστήμην ἢ τὸ τὴν ἡδονὴν μὴ εἶναι ἀγαθόν', 'Διαφέρει δὲ ἡ ἀποδεικτικὴ πρότασις τῆς διαλεκτικῆς, ὅτι ἡ μὲν ἀποδεικτικὴ λῆψις θατέρου μορίου τῆς ἀντιφάσεώς ἐστιν (οὐ γὰρ ἐρωτᾷ ἀλλὰ λαμβάνει ὁ ἁποδεικνύων), ἡ δὲ διαλεκτικὴ ἐρώτησις ἀντιφάσεώς ἐστιν', 'Οὐδὲν δὲ διοίσει πρὸς τὸ γενέσθαι τὸν ἑκατέρου συλλογισμόν', 'καὶ γὰρ ὁ ἀποδεικνύων καὶ 

In [0]:
sentences = []
for string in data_merged_raw[data_merged_raw["filename"].str.startswith("tlg0086.")]["string"].tolist()[:5]:
  sentences.extend(get_lemmatized_sentences(string, all_lemmata=True, filter_by_postag=["n", "a", "v"]))

In [0]:
print(sentences[:10])

[['πρότερος'], ['πρότερος', 'εἶπον', 'σκέψις', 'ἀπόδειξις', 'ἐπιστήμη'], ['διορίζω', 'πρότασις', 'ὅρος', 'συλλογισμός', 'ποῖος/ποιός', 'τέλειος', 'ποῖος/ποιός', 'ἀτελής', 'ὅλοξ/ὁλάω', 'λέγω', 'πᾶς', 'μηδείς', 'κατηγορέω'], ['πρότασις', 'λόγος'], ['μέρος'], ['λέγω', 'πᾶς', 'μηδείς', 'ὑπάρχω', 'μέρος', 'πᾶς', 'ὑπάρχω', 'ὑπάρχω', 'ὑπάρχω', 'ἀνίημι/ἄνω', 'μέρος', 'οἷος', 'ἐναντίος', 'ἐπιστήμη', 'ἀγαθός'], ['διαφέρω', 'πρότασις', 'διαλεκτικός', 'λῆψις', 'ἕτερος', 'μόριος/μόριον', 'ἐρωτάω', 'λαμβάνω', 'διαλεκτικός', 'ἐρώτησις'], ['διαφέρω', 'γίγνομαι', 'ἑκάτερος', 'συλλογισμός'], ['ἔρως/ἐρωτάω', 'συλλογίζομαι', 'λαβή', 'ὑπάρχω', 'ὑπάρχω'], ['πρότασις', 'ἁπλόω', 'τρόπος/τροπός', 'ἀρχή', 'ὑπόθεσις', 'διαλεκτικός', 'ἐρώτησις', 'λῆψις', 'ἔνδοξος', 'τοπικός', 'ἔρομαι/ἐρῶ/εἴρω']]


In [0]:
string_tokenized = []
for string in data_merged_raw[data_merged_raw["filename"].str.startswith("tlg0086.")]["string"].tolist():
  string_tokenized += tokenize_string(string)

In [0]:
string_tokenized[:100]

['ΑΝΑΛΥΤΙΚΩΝ',
 'ΠΡΟΤΕΡΩΝ',
 'ΠΡΩΤΟΝ',
 'εἰπεῖν',
 'περί',
 'τί',
 'καί',
 'τίνος',
 'ἐστίν',
 'σκέψις',
 'ὅτι',
 'περί',
 'ἀπόδειξιν',
 'καί',
 'ἐπιστήμης',
 'ἀποδεικτικῆς',
 'εἶτα',
 'διορίσαι',
 'τί',
 'ἐστι',
 'πρότασις',
 'καί',
 'τί',
 'ὅρος',
 'καί',
 'τί',
 'συλλογισμός',
 'καί',
 'ποῖος',
 'τέλειος',
 'καί',
 'ποῖος',
 'ἀτελής',
 'μετά',
 'δέ',
 'ταῦτα',
 'τί',
 'τό',
 'ἐν',
 'ὅλῳ',
 'εἶναι',
 'μη',
 'εἶναι',
 'τόδε',
 'τῷδε',
 'καί',
 'τί',
 'λέγομεν',
 'τό',
 'κατά',
 'παντός',
 'μηδενός',
 'κατηγορεῖσθαι',
 'Πρότασις',
 'μέν',
 'οὖν',
 'ἐστί',
 'λόγος',
 'καταφατικός',
 'ἀποφατικός',
 'τινός',
 'κατά',
 'τινος',
 'Οὗτος',
 'δέ',
 'καθόλου',
 'ἐν',
 'μέρει',
 'ἀδιόριστος',
 'Λέγω',
 'δέ',
 'καθόλου',
 'μέν',
 'τό',
 'παντί',
 'μηδενί',
 'ὑπάρχειν',
 'ἐν',
 'μέρει',
 'δέ',
 'τό',
 'τινί',
 'μη',
 'τινί',
 'μη',
 'παντί',
 'ὑπάρχειν',
 'ἀδιόριστον',
 'δέ',
 'τό',
 'ὑπάρχειν',
 'μη',
 'ὑπάρχειν',
 'ἄνευ',
 'τοῦ',
 'καθόλου',
 'κατά',
 'μέρος',
 'οἷον',
 'τό']

In [0]:
morpheus_dict["αἴτιος"]

[{'a': None,
  'b': 'αιτιος',
  'd': '950',
  'e': 'αιτιος',
  'f': 'αἴτιος',
  'i': '28485',
  'l': 'αἴτιος',
  'p': 'a-s---fn-',
  's': 'to blame, blameworthy, culpable'},
 {'a': None,
  'b': 'αιτιος',
  'd': '950',
  'e': 'αιτιος',
  'f': 'αἴτιος',
  'i': '28507',
  'l': 'αἴτιος',
  'p': 'a-s---mn-',
  's': 'to blame, blameworthy, culpable'}]

In [0]:
aristotle_lemmatized = []
all_string_tokenized = []
for string in data_merged_raw[data_merged_raw["filename"].str.startswith("tlg0086.")]["string"].tolist():
  string_tokenized = tokenize_string(string)
  string_tokenized = [word for word in string_tokenized if word not in STOPS_LIST]
  all_string_tokenized += string_tokenized

  #aristotle_lemmatized += lemmatizer.lemmatize(string_tokenized)

In [0]:
from cltk.tag.pos import POSTag
tagger = POSTag('greek')

In [0]:
postagged = tagger.tag_ngram_123_backoff(" ".join(all_string_tokenized))

In [0]:
len(postagged)

536816

In [0]:
### proportion of postagged words
len([tup for tup in postagged if tup[1] != None]) / len(postagged)

0.5629247265357218

In [0]:
len(postagged)

43

In [0]:
nltk.FreqDist(aristotle_lemmatized).most_common(100)

[('εἰμί', 32232),
 ('οὗτος', 13822),
 ('ὁ', 9785),
 ('αὐτός', 8172),
 ('ἔχω', 7257),
 ('πᾶς', 6549),
 ('γίγνομαι', 6093),
 ('ἄλλος', 4940),
 ('οἷος', 4824),
 ('λέγω1', 4664),
 ('ὑπάρχω', 4301),
 ('τις', 4016),
 ('πρότερος', 4007),
 ('ἵημι', 3508),
 ('ποιέω', 3201),
 ('οὐδείς', 3159),
 ('ὅς', 3095),
 ('τοιοῦτος', 2914),
 ('κινέω', 2892),
 ('ἕτερος', 2872),
 ('ὥσπερ', 2861),
 ('δέω1', 2739),
 ('Β', 2579),
 ('ἀνάγκη', 2449),
 ('συμβαίνω', 2389),
 ('οὐ', 2377),
 ('ὅσος', 2305),
 ('ἐρῶ', 2213),
 ('Α', 2170),
 ('ἕκαστος', 2154),
 ('ὁμοιόω', 2127),
 ('Γ', 2111),
 ('λόγος', 2090),
 ('φύω', 1976),
 ('μᾶλλον', 1967),
 ('ἐναντίος', 1954),
 ('ἐνδέχομαι', 1942),
 ('δοκέω', 1902),
 ('πολύς', 1890),
 ('ἑαυτοῦ', 1848),
 ('λαμβάνω', 1843),
 ('ἄνθρωπος', 1822),
 ('ὅλοξ', 1780),
 ('εἷς', 1758),
 ('ἀγαθός', 1685),
 ('μέγας', 1586),
 ('φημί', 1585),
 ('δῆλος', 1565),
 ('μόνος', 1559),
 ('φαίνω', 1555),
 ('ἀρχή', 1533),
 ('ἐπεί', 1524),
 ('γένος', 1513),
 ('ἀεί', 1491),
 ('μηδείς', 1460),
 ('διό', 1446),
 (

In [0]:
### Aristotle
string_lemmatized = []
for string in data_merged_raw[data_merged_raw["filename"].str.startswith("tlg0086.")]["string"].tolist():
  string_lemmatized += lemmatize_string(string, all_lemmata=True, filter_by_postag=["n","a","v"])
nltk.FreqDist(string_lemmatized).most_common(200)

[('πᾶς', 5366),
 ('γίγνομαι', 4870),
 ('οἷος', 4823),
 ('λέγω', 4048),
 ('πρότερος', 3969),
 ('ὑπάρχω', 3959),
 ('ἔχω', 3201),
 ('ἕτερος', 2872),
 ('ἵημι', 2630),
 ('λόγος', 2607),
 ('χέω/ἔχις/ἔχω', 2602),
 ('ποιέω', 2581),
 ('δεῖ/δέω', 2454),
 ('ὅσος', 2306),
 ('ὁμοιόω/ὅμοιος', 2168),
 ('ἕκαστος', 2154),
 ('εἰμί/εἶμι', 2016),
 ('μαλλός', 1969),
 ('πολύς', 1843),
 ('ἐναντίος', 1840),
 ('ἄνθρωπος', 1820),
 ('κινέω', 1814),
 ('ἀνάγκη', 1794),
 ('σῶμα', 1681),
 ('συμβαίνω', 1680),
 ('ἀγαθός', 1675),
 ('ἐνδέχομαι', 1647),
 ('μέγας', 1588),
 ('γένος', 1514),
 ('Δῆλος/δῆλος', 1508),
 ('ἔπειμι/ἔπος', 1462),
 ('μηδείς', 1460),
 ('δοκέω', 1452),
 ('φύσις', 1448),
 ('μέρος', 1407),
 ('φανερός', 1365),
 ('λαμβάνω', 1359),
 ('μόνος', 1330),
 ('γῆ', 1316),
 ('ὕδωρ', 1315),
 ('ὅλοξ', 1312),
 ('χρόνος', 1297),
 ('τρόπος/τροπός', 1280),
 ('συλλογισμός', 1267),
 ('ἀναγκαῖον/ἀναγκαῖος', 1234),
 ('εἰμί/ἐάω', 1185),
 ('φαίνω', 1157),
 ('ἀδύνατος', 1152),
 ('οὐσία', 1150),
 ('τόπος', 1150),
 ('δύναμις', 11

In [0]:
string_tokenized = []
for string in data_merged_raw[data_merged_raw["filename"].str.startswith("tlg0086.")]["string"].tolist():
  string_tokenized += tokenize_string(string)
nltk.FreqDist(string_tokenized).most_common(200)

[('καί', 51026),
 ('τό', 33541),
 ('δέ', 27198),
 ('γάρ', 18709),
 ('μέν', 15960),
 ('τῶν', 14956),
 ('τά', 10900),
 ('ἢ', 10870),
 ('τοῦ', 10655),
 ('ἐν', 10107),
 ('ἡ', 10037),
 ('την', 9299),
 ('τῷ', 9167),
 ('δʼ', 7810),
 ('εἶναι', 7540),
 ('μη', 6970),
 ('ὁ', 6718),
 ('τῆς', 6650),
 ('τοῖς', 6022),
 ('ὅτι', 5731),
 ('δ', 5683),
 ('εἰ', 5608),
 ('πρός', 5458),
 ('οὐ', 5310),
 ('διά', 5257),
 ('περί', 5209),
 ('τόν', 4911),
 ('οὖν', 4715),
 ('οὐκ', 4573),
 ('οἷον', 4459),
 ('κατά', 4419),
 ('οἱ', 4252),
 ('ἂν', 4068),
 ('τε', 3786),
 ('ἐπί', 3705),
 ('ὡς', 3699),
 ('τοῦτο', 3643),
 ('τι', 3557),
 ('ἐκ', 3464),
 ('ἀλλά', 3326),
 ('εἰς', 3225),
 ('τάς', 3037),
 ('ὥσπερ', 2792),
 ('ἐστιν', 2739),
 ('τοὺς', 2681),
 ('τῇ', 2647),
 ('ἔσται', 2628),
 ('αἱ', 2605),
 ('Β', 2579),
 ('τούτων', 2528),
 ('ἔχει', 2479),
 ('ἐξ', 2448),
 ('ἀλλʼ', 2395),
 ('ἐστίν', 2348),
 ('ἔστι', 2185),
 ('Α', 2170),
 ('Γ', 2111),
 ('δεῖ', 2099),
 ('τις', 2040),
 ('ὅταν', 1992),
 ('ταῦτα', 1972),
 ('μᾶλλον', 1956)

In [0]:
lemma_translator("διά")

'through c. gen. through, by means of c. acc.'

In [0]:
arist_most_common = [tup[0] for tup in nltk.FreqDist(string_lemmatized).most_common(1000)]


In [0]:
morpheus_by_lemma["κοινών"]

[{'a': None, 'd': 18272, 'l': 'κοινών', 's': None}]

In [0]:
lemma_translator("κοινών")

''

In [0]:
for word in arist_most_common:
  print([word, lemma_translator(word)])

['πᾶς', 'all, the whole']
['οἷος', 'such as, what sort']
['γίγνομαι', 'gi-geno']
['λέγω', 'to lay asleep, lull to sleep / to say, speak / to gather, pick up']
['πρότερος', 'prae.']
['ὑπάρχω', 'to begin, make a beginning']
['ἵημι', 'to set a going, put in motion']
['ἔχω', 'to have']
['ἕτερος', 'one of the']
['χέω/ἔχις/ἔχω', 'to pour / an adder, viper / to have']
['λόγος', 'the word']
['ποιέω', 'to make']
['δεῖ/δέω', 'to bind / to bind, tie, fetter / to lack, miss, stand in need of']
['ὅσος', 'quantus']
['ὁμοιόω/ὅμοιος', 'to make like / like, resembling']
['εἰμί/ἐάω/ἀάω', 'I have / to let, suffer, allow, permit / to hurt, damage']
['ἕκαστος', 'every, every one, each, each one']
['εἰμί/εἶμι', 'I have / to go']
['μαλλός', 'a lock of wool, wool']
['πολύς', 'many']
['κινέω', 'to set in motion, to move']
['ἀνάγκη', 'force, constraint, necessity']
['ἐναντίος', 'opposite']
['ἄνθρωπος', 'manfaced']
['σῶμα', 'the body']
['ἀγαθός', 'good']
['ἐνδέχομαι', 'to take upon oneself']
['συμβαίνω', 'to sta

In [0]:

for word in arist_most_common:
  for option in word.split("/"):
    for token in morpheus_by_lemma[option]:
      ([word, option, token["s"]])

In [0]:
"εἰμί/ἐάω/ἵημι/ἑός".split("/")

['εἰμί', 'ἐάω', 'ἵημι', 'ἑός']

In [0]:
### testing on a random subselection of strings
string_lemmatized = []
for number in random.choices(range(0, 1662), k=20):
  string = data_merged_raw.iloc[number]["string"]
  string_lemmatized += lemmatize_string(string, all_lemmata=True, filter_by_postag=None)
nltk.FreqDist(string_lemmatized).most_common(100)

[('λέγω', 4796),
 ('φημί', 4691),
 ('γίγνομαι', 3590),
 ('αὐτοῦ/αὐτός', 2826),
 ('λόγος', 2416),
 ('πᾶς', 2326),
 ('ἔχω', 2085),
 ('ποιέω', 1992),
 ('ἀλλ', 1970),
 ('εἶπον', 1927),
 ('τίς/τις', 1778),
 ('θεός', 1561),
 ('πρότερος', 1545),
 ('ἕτερος', 1514),
 ('ἄνθρωπος', 1476),
 ('λαμβάνω', 1428),
 ('θεάομαι/θεός', 1364),
 ('εἰμί/ἐάν/ἠμί/ἤν', 1351),
 ('ὅς/ὁ', 1290),
 ('ἁμοῦ/ἐμέω/ἐμός/ἁμός/ἡμός/ἐγώ', 1256),
 ('δείκνυμι', 1208),
 ('χριστός', 1202),
 ('ἔθω/καθά/καθό/κατά/εἴθε/εἶμι/μαράν/εἶτα/ἔτης/ἀτάομαι/ἄτη/ἐθάς/ἔτι/εἰμί/ἐτάζω/ἄατος/ἀίτας/εἴτε',
  1178),
 ('οἷος', 1175),
 ('πάλιν', 1141),
 ('γῆ', 1136),
 ('ὑπάρχω', 1115),
 ('ὥσπερ', 1093),
 ('ἐκεῖνος', 1049),
 ('συμβαίνω', 1046),
 ('ἤγουν', 1028),
 ('ἐπ', 1001),
 ('πολύς', 997),
 ('πρότασις', 997),
 ('δεῖ/δέω', 966),
 ('μέγας', 963),
 ('αὐτοὺς', 963),
 ('μᾶλλον/μαλλός', 941),
 ('ἀπόδειξις', 922),
 ('σεύω/σοῦ/σός', 911),
 ('λοιπός', 888),
 ('προφήτης', 875),
 ('ἐπειδη', 859),
 ('κύριος/κῦρος', 858),
 ('δίδωμι', 838),
 ('δοκέω', 835),
 ('μ

In [0]:
 ### testing for a random subselection of documents, print a random sentence
 for number in random.choices(range(0, 1662), k=30):
  string = data_merged_raw.iloc[number]["string"]
  sentences = get_lemmatized_sentences(string)
  print(sentences[random.choice(range(len(sentences)))])

['μέχρι', 'ὕδωρ', 'γράφω', 'κατακύπτω', 'πέτρα', 'ἵστημι', 'γράφω', 'εὐσχήμων', 'στέρνον', 'δείκνυμι', 'ἐπιπόλαιος', 'μίμησις', 'ὕδωρ', 'συνιζάνω', 'μῆκος']
['πάρειμι', 'βιόω', 'διʼ', 'ἔσοπτρος', 'αἰνίγμαει', 'πρόσωπον', 'πρόσωπον', 'ἀλήθεια', 'θεωρέω']
['θαυμάζω', 'μᾶλλον', 'ἐρῶ', 'ἐξαίφνης', 'ἀποδείκνυμι', 'φύω', 'δαιμονάω', 'ἐπιτηδεύω', 'ἀρετή', 'κἂν', '<εἰ>', 'λανθάνω', 'βραχὺ', 'λαμβάνω', 'πάλιν', 'ἀνα']
['περιειλήϲαντεϲ', 'διμερής', 'ϲειράν', 'ἄγω', 'περιϲϲοῦ', 'ἐπείληϲιν', 'ἐγκυκλέομαι', 'θώρακοϲ', 'προϲκαταλαμβανόμενοι', 'τοὺϲ', 'βραχίοναϲ', 'παρατεταγμένουϲ', 'ταῖϲ', 'πλευραῖϲ', 'ἵνα', 'άρμόϲῃ', 'κλείς', 'καταγειϲῶν']
['πλέω', 'οἷος', 'χροιά', 'ἀηρ', 'ὕδωρ', 'ἄμφω', 'διαφανής', 'ἕτερος', 'ἔχω', 'μόνος', 'αἰσθάνομαι', 'ἄμφω']
['ἕτερος', 'ἀπόεἶδον', 'τοιγάρ', 'ἀμείψομαί', 'φημί', 'δικαιοσύνη', 'καλός', 'δίκη', 'δικάζω', 'ἔρχομαι', 'ἦδος', 'ἐπιβαίνω', 'ὄχημα', 'δείξασά', 'ὄχημα', 'ὑπόπτερος', 'ἵππος', 'πήγασος', 'ἔοικα', 'ὅπως', 'οἶδα', 'οἷος', 'ἡλίκος', 'ἀκολουθέω', 'ἀγνοέω', 'μ

CPU times: user 4min 30s, sys: 1.69 s, total: 4min 32s
Wall time: 4min 32s


In [0]:
data_merged_raw.head(5)

,filename,author,title,string,lem_senteces
0,ggm0001.ggm001.1st1K-grc1.xml,Anonymous,Anametresis Pontou; Geographi Graeci Minores,ΑΝΩΝΥΜΟΥ ΑΝΑΜΕΤΡΗΣΙΣ ΤΗΣ ΟΙΚΟΥΜΕΝΗΣ ΗΑΣ ΗΕ ΚΑΤ...,"[[ανωνυμου, αναμετρησις, της, οικουμενης, ηας,..."
1,heb0001.heb010.1st1K-eng1.xml,None,Isaias; The Book of Isaiah,ISAIAH FROM HEBREW I. 1 The vision of Isaiah t...,"[[‘], [’], [‘], [‘], [‘], [‘], [’, ‘], [‘], [’..."
2,ogl0001.ogl001.1st1K-grc1.xml,Pinytus,De Epistola Pinyti ad Dionysium; Reliquiae sacrae,"FRAGMENTUM BEATI PINYTI, CNOSSI IN CRETA EPISC...","[[ἣν], [ἀντιπαρακαλέω], [δι᾿, ἐπιστολή, πινυτό..."
3,stoa0033a.tlg028.1st1K-grc1.xml,pseudo-Aristotle,De mundo; Aristotelis Opera,ΠΕΡΙ ΚΟΣΜΟΥ ΠΡΟΣ ΑΛΕΞΑΝΔΡΟΝ. ΠΟΛΛΑΚΙΣ μὲν ἔμοι...,"[[περι, κοσμου, προς, αλεξανδρον], [πολλακις, ..."
4,stoa0033a.tlg043.1st1K-grc1.xml,pseudo-Aristotle,De spiritu; Aristotelis Opera,ΠΕΡΙ ΠΝΕΥΜΑΤΟΣ. ΤΙΣ ἡ τοῦ ἐμφύτου πνεύματος δι...,"[[περι, πνευματος], [ἔμφυτος, πνεῦμα, διαμονή,..."


In [0]:
%%time
data_merged_raw["lemmatized"] = data_merged_raw.apply(lambda row: lemmatize_string(row["string"]), axis=1)

CPU times: user 4min 15s, sys: 1.34 s, total: 4min 16s
Wall time: 4min 16s


In [0]:
data_merged_raw.head(5)

,filename,author,title,string,lem_senteces,lemmatized
0,ggm0001.ggm001.1st1K-grc1.xml,Anonymous,Anametresis Pontou; Geographi Graeci Minores,ΑΝΩΝΥΜΟΥ ΑΝΑΜΕΤΡΗΣΙΣ ΤΗΣ ΟΙΚΟΥΜΕΝΗΣ ΗΑΣ ΗΕ ΚΑΤ...,"[[ανωνυμου, αναμετρησις, της, οικουμενης, ηας,...","[ανωνυμου, αναμετρησις, της, οικουμενης, ηας, ..."
1,heb0001.heb010.1st1K-eng1.xml,None,Isaias; The Book of Isaiah,ISAIAH FROM HEBREW I. 1 The vision of Isaiah t...,"[[‘], [’], [‘], [‘], [‘], [‘], [’, ‘], [‘], [’...","[ο, ο, ’, ?, ﬁ, ’, ‘, ’, . . . ., ‘, .’, . . ...."
2,ogl0001.ogl001.1st1K-grc1.xml,Pinytus,De Epistola Pinyti ad Dionysium; Reliquiae sacrae,"FRAGMENTUM BEATI PINYTI, CNOSSI IN CRETA EPISC...","[[ἣν], [ἀντιπαρακαλέω], [δι᾿, ἐπιστολή, πινυτό...","[. . . . ., ἣν, æ, πινυτός, ἀντιγράφω, θαυμάζω..."
3,stoa0033a.tlg028.1st1K-grc1.xml,pseudo-Aristotle,De mundo; Aristotelis Opera,ΠΕΡΙ ΚΟΣΜΟΥ ΠΡΟΣ ΑΛΕΞΑΝΔΡΟΝ. ΠΟΛΛΑΚΙΣ μὲν ἔμοι...,"[[περι, κοσμου, προς, αλεξανδρον], [πολλακις, ...","[περι, κοσμου, προς, αλεξανδρον., πολλακις, θε..."
4,stoa0033a.tlg043.1st1K-grc1.xml,pseudo-Aristotle,De spiritu; Aristotelis Opera,ΠΕΡΙ ΠΝΕΥΜΑΤΟΣ. ΤΙΣ ἡ τοῦ ἐμφύτου πνεύματος δι...,"[[περι, πνευματος], [ἔμφυτος, πνεῦμα, διαμονή,...","[περι, πνευματος., ἔμφυτος, πνεῦμα, διαμονή, α..."


# Inspecting Aristotle

In [0]:


len(data_merged_raw[data_merged_raw["filename"].str.startswith("tlg0086.")])

39

In [0]:
### inspecting Aristotle from a closer distance
data_merged_raw[data_merged_raw["filename"].str.startswith("tlg0086.")]

,filename,author,title,string,lem_senteces,lemmatized
156,tlg0086.tlg001.1st1K-grc1.xml,Aristotle,Analytica priora; Aristotelis Opera,ΑΝΑΛΥΤΙΚΩΝ ΠΡΟΤΕΡΩΝ Α. ΠΡΩΤΟΝ εἰπεῖν περὶ τί κ...,"[[αναλυτικων, προτερων, α], [πρωτον, εἶπον, σκ...","[αναλυτικων, προτερων, α., πρωτον, εἶπον, σκέψ..."
157,tlg0086.tlg001.1st1K-grc2.xml,Aristotle,Aristotelis Analytica Priora et Posteriora; Ar...,ΑΝΑΛΥΤΙΚΩΝ ΠΡΟΤΕΡΩΝ Α. Πρῶτον εἰπεῖν περὶ τί κ...,"[[αναλυτικων, προτερων, α], [πρότερος, εἶπον, ...","[αναλυτικων, προτερων, α., πρότερος, εἶπον, σκ..."
158,tlg0086.tlg002.1st1K-grc1.xml,Aristotle,De anima; Aristotelis Opera,ΠΕΡΙ ΨΥΧΗΣ Α. ΤΩΝ καλῶν καὶ τιμίων τὴν εἴδησιν...,"[[περι, ψυχης, α], [των, καλέω, τιμάω, εἴδομαι...","[περι, ψυχης, α., των, καλέω, τιμάω, εἴδομαι, ..."
159,tlg0086.tlg002.1st1K-grc2.xml,Aristotle,De anima; Aritoteles De anima,ΠEP1 ΨΥΧΗΣ Α Τῶν καλῶν καὶ τιμίων τὴν εἴδησιν ...,"[[π], [δοκέω, ἀλήθεια, ἅπας, γνῶσις, μέγας, συ...","[π, ψυχης, α, καλέω, τιμάω, εἴδομαι, ὑπολαμβάν..."
160,tlg0086.tlg005.1st1K-grc1.xml,Aristotle,De caelo; Aristoteles De coelo et De generatio...,ΠEPI ΟΥΡΑΝΟΥ A. Ἡ περὶ φύσεως ἐπιστήμη σχεδὸν ...,"[[π], [φύσις, ἐπιστήμη, σχεδόν, πλεῖστος, φαίν...","[π, ουρανου, φύσις, ἐπιστήμη, σχεδόν, πλεῖστος..."
161,tlg0086.tlg006.1st1K-grc1.xml,Aristotle,Categoriae; Aristotelis Opera,ΚΑΤΗΓΟΡΙΑΙ. ὉΜΩΝΥΜΑ λέγεται ὧν ὄνομα μόνον κοι...,"[[κατηγοριαι], [ὁμωνυμα, λέγω, ὄνομα, μόνος, κ...","[κατηγοριαι., ὁμωνυμα, λέγω, ὄνομα, μόνος, κοι..."
162,tlg0086.tlg008.1st1K-grc1.xml,Aristotle,De divinatione per somnum; Aristotelis Opera,ΠΕΡΙ ΕΝΥΠΝΙΩΝ ΚΑΙ ΤΗΣ ΚΑΘ’ ΥΠΝΟΝ ΜΑΝΤΙΚΗΣ. Περ...,"[[περι, ενυπνιων, και, της, υπνον, μαντικης], ...","[περι, ενυπνιων, και, της, υπνον, μαντικης., μ..."
163,tlg0086.tlg013.1st1K-grc1.xml,Aristotle,De generatione et corruptione; Aristotle. On S...,ΑΙΣΤΟΤΕΛΟΥΣ ΠΕΡΠ ΓΕΝΕΣΕΩΣ ΚΑΙ ΦΘΟΡΑΣ Α 314 a 1...,"[[αιστοτελους, περπ, γενεσεως, και, φθορας, α]...","[αιστοτελους, περπ, γενεσεως, και, φθορας, α, ..."
164,tlg0086.tlg013.1st1K-grc2.xml,Aristotle,De generatione et corruptione; Aristoteles De ...,ΠΕΡΙ ΙΕΝΕΣΕΩΣ ΚΑΙ ΦΘΟΡΑΣ Α. Περὶ δὲ γενέσεως κ...,"[[περι, ιενεσεως, και, φθορας, α], [γένεσις, φ...","[περι, ιενεσεως, και, φθορας, α., γένεσις, φθο..."
165,tlg0086.tlg014.1st1K-grc1.xml,Aristotle,Historia animalium; Aristotelis Opera,ΠΕΡΙ ΤΑ ΖΩΙΑ ΙΣΤΟΡΙΩΝ Α. ΤΩΝ ἐν τοῖς ζῴοις μορ...,"[[περι, τα, ζωια, ιστοριων, α], [των, ζώιον, μ...","[περι, τα, ζωια, ιστοριων, α., των, ζώιον, μορ..."


In [0]:
arist = data_merged_raw[data_merged_raw["filename"].str.startswith("tlg0086.tlg010")]["lem_senteces"].tolist()[0]
print(arist)

[['βοοκ', 'πᾶς', 'τέχνη', 'καί', 'πᾶς', 'μέθοδος', 'ὁμοιόω', 'δέ', 'πρᾶξίς', 'καί', 'προαίρεσις', 'ἀγαθός', 'ἐφίημι', 'δοκέω'], ['διό', 'καλός', 'ἀποφαίνω', 'ἀγαθός', 'ἵημι', 'πάντʼ', 'ἐφίημι'], ['διαφορά', 'δέ', 'φαίνω', 'τελέω'], ['μέν', 'γάρ', 'εἰμί', 'ἐνέργεια', 'δέ', 'παρʼ', 'αὐτός', 'ἔργον'], ['δʼ', 'εἰμί', 'τελέω', 'πηρός', 'πράσσω', 'βελτιόω', 'φύω', 'ἐνέργεια', 'ἔργον'], ['πολύς', 'δέ', 'πρᾶξις', 'εἰμί', 'καί', 'τεχνάζω', 'καί', 'ἐπιστήμη', 'πολύς', 'γίγνομαι', 'καί', 'τελέω'], ['ἰατρικός', 'μέν', 'γάρ', 'ὑγίεια', 'ναυπηγικός', 'δέ', 'πλοῖον', 'στρατηγικός', 'δέ', 'νικάω', 'οἰκονομικός', 'δέ', 'πλοῦτος'], ['ὅσος', 'δʼ', 'εἰμί', 'τοιοῦτος', 'ὑπό', 'εἷς', 'δύναμις', 'καθά', 'ὑπό', 'την', 'ἱππικην', 'χαλινοποιικη', 'καί', 'ὅσος', 'ἱππικός', 'ὀργανόω', 'εἰμί', 'δέ', 'καί', 'πᾶς', 'πολεμικη', 'πρᾶξις', 'ὑπό', 'την', 'στρατηγικός', 'κατά', 'αὐτός', 'δη', 'τρόπος', 'ὑφʼ', 'ἕτερος'], ['ἅπας', 'δέ', 'ἀρχιτεκτονικός', 'τελέω', 'πᾶς', 'εἰμί', 'αἱρετός', 'ὑπʼ', 'αὐτός'], ['γάρ', 'χάρις', 

In [0]:
data_merged_raw.to_json("temp.json")

In [0]:
s.put(sddk_url + "SDAM_data/OGL/data_lemmatized.json",  data=data_merged_raw.to_json())

OverflowError: ignored